# The detailed demonstration of the tropical rainfall diagnostic for low-resolution data:

# Calculation of histograms

### Content


1. [The load of packages, modules, and data](#1-load-of-packages-modules-and-data)

2. [Calculation of histogram of precipitation/tropical precipitation](#2-calculation-of-histogram-of-precipitationtropicalprecipitation)

    2.1. [with continuous  uniform binning](#21-with-continuous-uniform-binning)

    2.2  [with non-uniform (log-spaced) binning](#22-with-non-uniform-log-spaced-binning)
    
    2.3. [with non-default tropical latitude band](#24-with-non-default-latitude-band)

    2.4  [with a specific time band](#25-with-a-specific-time-band)

    2.5. [with weights (weights=reader.grid_area)](#26-with-weights-weightsreadergrid_area)
 

3. [Loading the histogram to/from storage](#3-loading-the-histogram-tofrom-storage)

[Go to the end of file](#the-end)

#

## 1. The load of packages, modules, and data

In [33]:
from aqua import Reader
from tropical_rainfall import Tropical_Rainfall

#### ICON data

In [34]:
reader = Reader(model="ICON", exp="historical-1990", source="lra-r100-monthly")
icon = reader.retrieve() 

2024-04-16 20:50:32 :: Reader :: WARNING  -> LRA_LUMI parameter is required but is missing, setting to default /work/bb1153/b382076/LRA-lumi


/work/bb1153/b382267/mamba_dir/mambaforge/envs/aqua/lib/python3.11/site-packages/dask/array/core.py:4836: PerformanceWarning: Increasing number of chunks by factor of 72
  result = blockwise(
/work/bb1153/b382267/mamba_dir/mambaforge/envs/aqua/lib/python3.11/site-packages/dask/array/core.py:4836: PerformanceWarning: Increasing number of chunks by factor of 72
  result = blockwise(


## 2. Calculation of histogram of tropical precipitation


### 2.1. with continuous uniform binning 

##### We can perform calculations of the histograms with uniform binning by  initializing the following attributes of diagnostic: 
 - `num_of_bins`
 - `first_edge`
 - `width_of_bin`
 
 ##### In this case, all the bins in the histogram will have the width of bins.

In [35]:
diag = Tropical_Rainfall(num_of_bins = 20, first_edge = 0, width_of_bin = 2)

##### We calculate the histogram simply by function:

In [36]:
hist_icon_trop  = diag.histogram(icon)
hist_icon_trop

2024-04-16 20:50:38 :: Trop. Rainfall :: WARNING  -> Amount of counts in the histogram is not equal to the size of the data
2024-04-16 20:50:38 :: Trop. Rainfall :: WARNING  -> Check the data and the bins


<xarray.Dataset> Size: 960B
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 160B 1.0 3.0 5.0 ... 35.0 37.0 39.0
    width          (center_of_bin) float64 160B 2.0 2.0 2.0 2.0 ... 2.0 2.0 2.0
Data variables:
    counts         (center_of_bin) float64 160B 8.643e+05 2.572e+05 ... 244.0
    frequency      (center_of_bin) float64 160B 0.4062 0.1209 ... 0.0001147
    pdf            (center_of_bin) float64 160B 0.2031 0.06044 ... 5.734e-05
    pdfP           (center_of_bin) float64 160B 0.2031 0.1813 ... 0.002236
Attributes:
    history:                \n2024-04-16 20:50:37 AQUA💧: Retrieved from ICON_...
    time_band:              1990-01-01T00:00:00.000000000, 2006-05-01T00:00:0...
    lat_band:               -14.5, 14.5, freq=1.0
    lon_band:               0.0, 359.0, freq=1.0
    time_band_history:      1990-01-01T00:00:00.000000000, 2006-05-01T00:00:0...
    units:                  mm/day
    mean_of_original_data:  5.143757961054234
    mean_of_histogram:      5.338574920097762
    relative_discrepancy:   -3.7874441316752594

##### The output of the histogram function is xarray.Dataset, which has two coordinates 
- `center_of_bin`:   the center of each bin
- `width`:           width of each bin
##### and contains three variables:
- `counts`:       the number of observations that fall into each bin
- `frequency`:    the number of cases in each bin, normalized by the total number of counts. The sum of the frequencies equals 1.
- `pdf`:          the number of cases in each bin, normalized by the total number of counts and width of each bin. 

##### local and global attributes. Local attributes contain the information about the time and space grid for which diagnostic performed the calculations:
- `time_band`:    the value of time of the first and last element in the dataset and the frequency of the time grid
- `lat_band`:     the maximum and minimum values of the tropical latitude band and the frequency of the latitude grid
- `lon_band`:     the maximum and minimum values of the longitude and the frequency of the longitude grid

##### Global attribute `history` contains the information about when the histogram was calculated and values of `time_band`, `lat_band`, and `lon_band`.

### 2.2.  with non-uniform (log-spaced) binning  

##### If you want to perform the calculation for non-uniform binning, use the `bins` attribute of the diagnostic instead of  `num_of_bins`, `first_edge`, and `width_of_bin`.  

In [37]:
bins = [1.00000000e-09, 1.63789371e-09, 2.68269580e-09, 4.39397056e-09,
       7.19685673e-09, 1.17876863e-08, 1.93069773e-08, 3.16227766e-08,
       5.17947468e-08, 8.48342898e-08, 1.38949549e-07, 2.27584593e-07,
       3.72759372e-07, 6.10540230e-07, 1.00000000e-06]

In [38]:
diag = Tropical_Rainfall(bins=bins)

In [39]:
hist_icon_trop_logspaced  = diag.histogram(icon)
hist_icon_trop_logspaced

2024-04-16 20:50:38 :: Trop. Rainfall :: WARNING  -> Amount of counts in the histogram is not equal to the size of the data
2024-04-16 20:50:38 :: Trop. Rainfall :: WARNING  -> Check the data and the bins
2024-04-16 20:50:38 :: Trop. Rainfall :: WARNING  -> The difference between the mean of the data and the mean of the histogram is greater                                 than the threshold. 
 Increase the number of bins and decrease the width of the bins.


<xarray.Dataset> Size: 672B
Dimensions:        (center_of_bin: 14)
Coordinates:
  * center_of_bin  (center_of_bin) float64 112B 1.319e-09 2.16e-09 ... 8.053e-07
    width          (center_of_bin) float64 112B 6.379e-10 ... 3.895e-07
Data variables:
    counts         (center_of_bin) int64 112B 4 7 12 32 21 ... 79 92 144 190 326
    frequency      (center_of_bin) float64 112B 1.88e-06 3.29e-06 ... 0.0001532
    pdf            (center_of_bin) float64 112B 2.947e+03 3.149e+03 ... 393.4
    pdfP           (center_of_bin) float64 112B 3.887e-06 ... 0.0003168
Attributes:
    history:                \n2024-04-16 20:50:37 AQUA💧: Retrieved from ICON_...
    time_band:              1990-01-01T00:00:00.000000000, 2006-05-01T00:00:0...
    lat_band:               -14.5, 14.5, freq=1.0
    lon_band:               0.0, 359.0, freq=1.0
    time_band_history:      1990-01-01T00:00:00.000000000, 2006-05-01T00:00:0...
    units:                  mm/day
    mean_of_original_data:  5.143757961054234
    mean_of_histogram:      2.0513184012947452e-10
    relative_discrepancy:   99.99999999601202

### 2.3. with non-default latitude band 

##### The tropical latitude band, by default, is the following

In [40]:
diag.trop_lat

15

##### You can modify the tropical latitude band as 

In [41]:
diag.trop_lat = 20
diag.trop_lat

20

##### You can also modify the latitude band with the use of an argument `trop_lat` of the histogram function 

In [42]:
hist_icon_trop  = diag.histogram(icon, trop_lat=30)
hist_icon_trop

2024-04-16 20:50:38 :: Trop. Rainfall :: WARNING  -> Amount of counts in the histogram is not equal to the size of the data
2024-04-16 20:50:38 :: Trop. Rainfall :: WARNING  -> Check the data and the bins
2024-04-16 20:50:39 :: Trop. Rainfall :: WARNING  -> The difference between the mean of the data and the mean of the histogram is greater                                 than the threshold. 
 Increase the number of bins and decrease the width of the bins.


<xarray.Dataset> Size: 672B
Dimensions:        (center_of_bin: 14)
Coordinates:
  * center_of_bin  (center_of_bin) float64 112B 1.319e-09 2.16e-09 ... 8.053e-07
    width          (center_of_bin) float64 112B 6.379e-10 ... 3.895e-07
Data variables:
    counts         (center_of_bin) int64 112B 10 47 43 106 ... 454 554 711 1268
    frequency      (center_of_bin) float64 112B 2.35e-06 1.105e-05 ... 0.000298
    pdf            (center_of_bin) float64 112B 3.684e+03 1.057e+04 ... 765.1
    pdfP           (center_of_bin) float64 112B 4.859e-06 ... 0.0006161
Attributes:
    history:                \n2024-04-16 20:50:37 AQUA💧: Retrieved from ICON_...
    time_band:              1990-01-01T00:00:00.000000000, 2006-05-01T00:00:0...
    lat_band:               -29.5, 29.5, freq=1.0
    lon_band:               0.0, 359.0, freq=1.0
    time_band_history:      1990-01-01T00:00:00.000000000, 2006-05-01T00:00:0...
    units:                  mm/day
    mean_of_original_data:  3.963323455159577
    mean_of_histogram:      3.9867073686362804e-10
    relative_discrepancy:   99.99999998994099

### 2.4. with a specific time band 

##### You can specify `s_time` and `f_time` as integers (starting and final times). For example, 

In [43]:
diag = Tropical_Rainfall(trop_lat = 30,  num_of_bins = 20, first_edge = 0, 
                         width_of_bin = 55, s_time = 0, f_time = 47)

In [44]:
hist_icon = diag.histogram(icon)
hist_icon

<xarray.Dataset> Size: 960B
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 160B 27.5 82.5 ... 1.072e+03
    width          (center_of_bin) float64 160B 55.0 55.0 55.0 ... 55.0 55.0
Data variables:
    counts         (center_of_bin) float64 160B 1.015e+06 60.0 0.0 ... 0.0 0.0
    frequency      (center_of_bin) float64 160B 0.9999 5.91e-05 0.0 ... 0.0 0.0
    pdf            (center_of_bin) float64 160B 0.01818 1.075e-06 ... 0.0 0.0
    pdfP           (center_of_bin) float64 160B 0.5 8.865e-05 0.0 ... 0.0 0.0
Attributes:
    history:                \n2024-04-16 20:50:37 AQUA💧: Retrieved from ICON_...
    time_band:              1990-01-01T00:00:00.000000000, 1993-11-01T00:00:0...
    lat_band:               -29.5, 29.5, freq=1.0
    lon_band:               0.0, 359.0, freq=1.0
    time_band_history:      1990-01-01T00:00:00.000000000, 1993-11-01T00:00:0...
    units:                  mm/day
    mean_of_original_data:  3.984024695655807
    mean_of_histogram:      27.503250591016545
    relative_discrepancy:   -590.3383561102464

##### Also, you can specify `s_time` and `f_time` as strings. For example, 

In [45]:
diag.s_time = '2020:01'
diag.f_time ='2020/03/20/12'

##### or

In [46]:
diag.s_time = '2020'
diag.f_time ='2020.03.20'

##### There is the possibility of specifying only the year band or only the months' band. For example, we can select June, July, and August in a whole dataset as

In [47]:
diag.s_month = 3
diag.f_month = 6 

In order to evaluate the original dataset, we reset the time band at the full time range available in the dataset:

In [48]:
diag.s_time = None
diag.f_time = None
diag.s_month = None
diag.f_month = None

### 2.5. with weights (weights=reader.grid_area)

##### The function provides the opportunity to calculate the histogram with weights. Compared to standard methods, such computations are `high-speed` because they are based on `dask_histogram` package.

In [49]:
diag = Tropical_Rainfall(num_of_bins = 20, first_edge = 0, width_of_bin = 6, 
                         s_time = 0, f_time = 47)

In [50]:
hist_icon_trop_weighted  = diag.histogram(icon, weights=reader.grid_area)
hist_icon_trop_weighted

<xarray.Dataset> Size: 960B
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 160B 3.0 9.0 15.0 ... 111.0 117.0
    width          (center_of_bin) float64 160B 6.0 6.0 6.0 6.0 ... 6.0 6.0 6.0
Data variables:
    counts         (center_of_bin) float64 160B 3.182e+05 1.302e+05 ... 0.0 0.0
    frequency      (center_of_bin) float64 160B 0.6268 0.2565 ... 0.0 0.0
    pdf            (center_of_bin) float64 160B 0.1045 0.04275 ... 0.0 0.0
    pdfP           (center_of_bin) float64 160B 0.3134 0.3847 0.2184 ... 0.0 0.0
Attributes:
    history:                \n2024-04-16 20:50:37 AQUA💧: Retrieved from ICON_...
    time_band:              1990-01-01T00:00:00.000000000, 1993-11-01T00:00:0...
    lat_band:               -14.5, 14.5, freq=1.0
    lon_band:               0.0, 359.0, freq=1.0
    time_band_history:      1990-01-01T00:00:00.000000000, 1993-11-01T00:00:0...
    units:                  mm/day
    mean_of_original_data:  5.2439751084711395
    mean_of_histogram:      6.182423167848699
    relative_discrepancy:   -17.895738251342706

#

## 3. Loading the histogram to/from storage

#### Saving the histogram in storage. 

##### `Notice`: the name of the file will be unique and depends on the time band 

In [51]:
path_to_histogram='/work/bb1153/b382267/AQUA/histograms/' # please change this path to your own path
filename = diag.dataset_to_netcdf(dataset = hist_icon, path_to_netcdf = path_to_histogram, 
                                  name_of_file = 'icon')

#### Loading/opening the histogram from the storage 

##### The information about how to open and merge the list of histograms, see the notebook  `diagnostic_vs_streaming.ipynb`

In [52]:
diag.tools.open_dataset(path_to_netcdf = filename)

<xarray.Dataset> Size: 960B
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 160B 27.5 82.5 ... 1.072e+03
    width          (center_of_bin) float64 160B ...
Data variables:
    counts         (center_of_bin) float64 160B ...
    frequency      (center_of_bin) float64 160B ...
    pdf            (center_of_bin) float64 160B ...
    pdfP           (center_of_bin) float64 160B ...
Attributes:
    history:                \n2024-04-16 20:48:28 AQUA💧: Retrieved from ICON_...
    time_band:              1990-01-01T00:00:00.000000000, 1993-11-01T00:00:0...
    lat_band:               -29.5, 29.5, freq=1.0
    lon_band:               0.0, 359.0, freq=1.0
    time_band_history:      1990-01-01T00:00:00.000000000, 1993-11-01T00:00:0...
    units:                  mm/day
    mean_of_original_data:  3.984024695655807
    mean_of_histogram:      27.503250591016545
    relative_discrepancy:   -590.3383561102464

[Back to the top of file](#structure)

##### The end